In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from datetime import date


SEED = 42

df_treino = pd.read_csv('dados/train.csv', index_col='id')
df_teste = pd.read_csv('dados/test.csv', index_col='id')

def feature_engeneering(df):
    df_eng = df.copy()

    # --- LIMPEZA INICIAL ---
    df_eng = df_eng.drop_duplicates()

    df_eng['hp'] = df_eng['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)
    df_eng['liters'] = df_eng['engine'].str.extract(r'(\d+\.?\d*)\s?L').astype(float)

    # --- 2. Idade e Uso ---
    var_ano_atual = date.today().year
    df_eng['car_age'] = var_ano_atual - df_eng['model_year']
    df_eng['car_age'] = df_eng['car_age'].replace(0, 1)
        
    # Agora a divisão é segura
    df_eng['miles_p_year'] = df_eng['milage'] / df_eng['car_age']
    

    # --- FUEL TYPE ---
    def clean_fuel(val):
        s = str(val).lower()
        if 'hybrid' in s:
            return 'Hybrid'
        elif 'not supported' in s:
            return 'EV'
        elif 'not supported' in s:
            return 'EV'
        else:
            return val

    df_eng['fuel_type'] = df_eng['fuel_type'].apply(clean_fuel)

    # ---TRANSMISSION TYPE ---
    def clean_transmission(val):
        s = str(val).lower()
        if 'automatic' in s or 'a/t' in s or 'cvt' in s:
            return 'Automatico'
        elif 'manual' in s or 'm/t' in s:
            return 'Manual'
        else:
            return 'Outro'
            
    df_eng['transmission_type'] = df_eng['transmission'].apply(clean_transmission)
    
    top_ext_colors = df_eng['ext_col'].value_counts().nlargest(10).index

    def simplificar_cor_ext(cor):
        if cor in top_ext_colors:
            return cor
        return 'Other'

    df_eng['ext_col_simple'] = df_eng['ext_col'].apply(simplificar_cor_ext)
    
    # Cores Interiores
    top_int_colors = df_eng['int_col'].value_counts().nlargest(10).index
    
    def simplificar_cor_int(cor):
        if cor in top_int_colors:
            return cor
        return 'Other'

    df_eng['int_col_simple'] = df_eng['int_col'].apply(simplificar_cor_int)

    # --- 4. Tratamento de Nulos ---
    
    cols_texto = df_eng.select_dtypes(include=['object']).columns
    df_eng[cols_texto] = df_eng[cols_texto].replace('-', 'Unknown').fillna('Unknown')
    
    df_eng['clean_title'] = df_eng['clean_title'].replace('Unknown', 'No')

    # --- 5. Acidente (Substituição do Lambda) ---
    
    def verificar_acidente(valor):
        s = str(valor)
        if 'None' in s:
            return 0
        return 1

    df_eng['accident_clean'] = df_eng['accident'].apply(verificar_acidente)
    
    return df_eng

X_train_processed = feature_engeneering(df_treino)
X_test_processed = feature_engeneering(df_teste)

# -------------------------------------------------------------------------
# 3. Preparação para o Modelo (Sem Pipelines)
# -------------------------------------------------------------------------

# A. Tratamento do Target (Log Transform)
# Importante: O modelo vai prever log(preço). Melhora muito o RMSE para preços altos.
y_train = np.log1p(df_treino['price'])  

# B. Seleção de Colunas
cat_cols = ['brand', 'fuel_type', 'transmission_type', 'ext_col_simple', 'int_col_simple', 'clean_title']
num_cols = ['milage', 'car_age', 'hp', 'liters', 'miles_p_year', 'accident_clean']

# C. Imputação de Numéricos (Fit no treino, Transform no teste)
# Preencher HP e Litros vazios com a mediana do treino
for col in ['hp', 'liters']:
    median_val = X_train_processed[col].median()
    X_train_processed[col] = X_train_processed[col].fillna(median_val)
    X_test_processed[col] = X_test_processed[col].fillna(median_val)

# D. One-Hot Encoding Manual (Pandas get_dummies)
# Concatenamos para garantir as mesmas colunas, depois separamos
train_len = len(X_train_processed)
combined = pd.concat([X_train_processed[cat_cols], X_test_processed[cat_cols]], axis=0)
combined_dummies = pd.get_dummies(combined, drop_first=True)

# Separar de volta
X_train_cat = combined_dummies.iloc[:train_len]
X_test_cat = combined_dummies.iloc[train_len:]

# Juntar Numéricos e Categóricos
X_train_final = pd.concat([X_train_processed[num_cols], X_train_cat], axis=1)
X_test_final = pd.concat([X_test_processed[num_cols], X_test_cat], axis=1)

print(f"Features finais: {X_train_final.shape[1]}")

# -------------------------------------------------------------------------
# 4. Grid Search e Treino (XGBoost)
# -------------------------------------------------------------------------

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=SEED,
    n_jobs=-1
)

param_grid = {
    'n_estimators': [300, 500],
    'learning_rate': [0.01, 0.05],
    'max_depth': [5, 7],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9]
}

print("A iniciar Grid Search (pode demorar um pouco)...")
grid = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    verbose=1
)

grid.fit(X_train_final, y_train)

print(f"Melhores parâmetros: {grid.best_params_}")
# O score aqui é em escala logarítmica, difícil de interpretar diretamente
print(f"Melhor RMSE (Log Scale): {-grid.best_score_:.4f}")

# -------------------------------------------------------------------------
# 5. Avaliação Final e Gráficos
# -------------------------------------------------------------------------
best_model = grid.best_estimator_

# Previsões no Treino (para gráfico)
y_pred = best_model.predict(X_train_final)
y_pred_real = np.expm1(y_pred) # Reverter log para preço real
y_train_real = np.expm1(y_train)

# Calcular RMSE real no treino
rmse_train = np.sqrt(mean_squared_error(y_train_real, y_pred_real))
print(f"RMSE no Treino (Escala Real): {rmse_train:.2f}")

# Gráfico Real vs Previsto
plt.figure(figsize=(10, 6))
plt.scatter(y_train_real, y_pred_real, alpha=0.5, color='blue')
plt.plot([y_train_real.min(), y_train_real.max()], [y_train_real.min(), y_train_real.max()], 'r--', lw=2)
plt.xlabel('Preço Real ($)')
plt.ylabel('Preço Previsto ($)')
plt.title(f'XGBoost: Real vs Previsto (RMSE Treino: {rmse_train:.0f})') #com Log-Target
#plt.xscale('log') # Escala log ajuda a visualizar melhor a dispersão
#plt.yscale('log')
plt.show()

# Feature Importance
importances = best_model.feature_importances_
feature_names = X_train_final.columns
feat_imp = pd.DataFrame({'feature': feature_names, 'importance': importances})
feat_imp = feat_imp.sort_values('importance', ascending=False).head(20)

plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=feat_imp, palette='viridis')
plt.title('Top 20 Features Mais Importantes')
plt.show()

# -------------------------------------------------------------------------
# 6. Submissão
# -------------------------------------------------------------------------
# Prever no teste
test_pred_log = best_model.predict(X_test_final)
test_pred_real = np.expm1(test_pred_log) # IMPORTANTE: Reverter o log!

submission = pd.DataFrame({
    'id': df_teste.index,
    'price': test_pred_real
})

submission.to_csv('submission.csv', index=False)
print("Ficheiro '.csv' gerado com sucesso.")

Features finais: 91
A iniciar Grid Search (pode demorar um pouco)...
Fitting 5 folds for each of 32 candidates, totalling 160 fits



KeyboardInterrupt


KeyboardInterrupt

